## Imports

In [31]:
import numpy as np
import pandas as pd
import re
import codecs
import string
from Process import *
import tensorflow as tf

## Read input

In [12]:
english_path = '../Data/IITB.en-hi.en'
hindi_path =  '../Data/IITB.en-hi.hi'

In [37]:
def read_data(filepath,lines):
    
        """
        filepath is address of file
        lines == All will read complete document else specified number of lines
            """
        if lines=='all':
            with codecs.open(filepath,encoding='UTF-8') as f:
                data = f.read().split('\n')
            
        else:
            with codecs.open(filepath,encoding='UTF-8') as f:
                data = f.readlines(lines)
        return data

In [38]:
english = read_data(english_path,lines='all')
hindi = read_data(hindi_path,lines='all')

In [39]:
for i in range(15):
    print("English:",english[i])
    print("Hindi:",hindi[i])
    print('\n\n')

English: Give your application an accessibility workout
Hindi: अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें



English: Accerciser Accessibility Explorer
Hindi: एक्सेर्साइसर पहुंचनीयता अन्वेषक



English: The default plugin layout for the bottom panel
Hindi: निचले पटल के लिए डिफोल्ट प्लग-इन खाका



English: The default plugin layout for the top panel
Hindi: ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका



English: A list of plugins that are disabled by default
Hindi: उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है



English: Highlight duration
Hindi: अवधि को हाइलाइट रकें



English: The duration of the highlight box when selecting accessible nodes
Hindi: पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि



English: Highlight border color
Hindi: सीमांत (बोर्डर) के रंग को हाइलाइट करें



English: The color and opacity of the highlight border.
Hindi: हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। 



English: Highlight fill color
Hindi: भराई के रंग को हाइलाइट करें



Engl

In [40]:
len(english),len(hindi)

(1659084, 1659084)

## Preprocessing

In [41]:
def preprocess(text):
  text = ''.join(ch for ch in text if ch not in string.punctuation)
  text = text.lower()
  text = re.sub(r'\d','',text)
  text = re.sub(r'\s+',' ',text)  #Remove extra spaces
  text = text.strip()
  return text

In [42]:
english_sentences = [preprocess(text) for text in english]

hindi_sentences = [preprocess(text) for text in hindi]

In [43]:
english_sentences[:10]

['give your application an accessibility workout',
 'accerciser accessibility explorer',
 'the default plugin layout for the bottom panel',
 'the default plugin layout for the top panel',
 'a list of plugins that are disabled by default',
 'highlight duration',
 'the duration of the highlight box when selecting accessible nodes',
 'highlight border color',
 'the color and opacity of the highlight border',
 'highlight fill color']

In [44]:
hindi_sentences[:10]

['अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें',
 'एक्सेर्साइसर पहुंचनीयता अन्वेषक',
 'निचले पटल के लिए डिफोल्ट प्लगइन खाका',
 'ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका',
 'उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है',
 'अवधि को हाइलाइट रकें',
 'पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से की अवधि',
 'सीमांत बोर्डर के रंग को हाइलाइट करें',
 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।',
 'भराई के रंग को हाइलाइट करें']

In [45]:
len(english_sentences),len(hindi_sentences)

(1659084, 1659084)

In [46]:
# constituting English words with space in hindi 
hindi_sentences = [re.sub(r'[a-zA-Z]','',hi) for hi in hindi_sentences]

In [55]:
#Some parameters
vocab_size = 10000
total_sentences = 50000
maxlen = 10
epochs = 70
validation_split = 0.05

In [54]:


en_data = []
hi_data = []

cnt = 0

for (en,hi) in zip(english_sentences, hindi_sentences):
  l = min(len(en.split()), len(hi.split()))
  if l <= maxlen:
    en_data.append(en)
    hi_data.append(hi)
    cnt += 1
  if cnt == total_sentences:
    break

In [56]:
en_data

['give your application an accessibility workout',
 'accerciser accessibility explorer',
 'the default plugin layout for the bottom panel',
 'the default plugin layout for the top panel',
 'a list of plugins that are disabled by default',
 'highlight duration',
 'the duration of the highlight box when selecting accessible nodes',
 'highlight border color',
 'the color and opacity of the highlight border',
 'highlight fill color',
 'the color and opacity of the highlight fill',
 'api browser',
 'browse the various methods of the current accessible',
 'hide private attributes',
 'method',
 'property',
 'value',
 'ipython console',
 'interactive console for manipulating currently selected accessible',
 'event monitor',
 'monitor events',
 'c lear selection',
 'everything',
 'selected application',
 'selected accessible',
 'source',
 'event monitor',
 'shows events as they occur from selected types and sources',
 'highlight last event entry',
 'start stop event recording',
 'clear event lo

In [57]:
hi_data

['अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें',
 'एक्सेर्साइसर पहुंचनीयता अन्वेषक',
 'निचले पटल के लिए डिफोल्ट प्लगइन खाका',
 'ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका',
 'उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है',
 'अवधि को हाइलाइट रकें',
 'पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से की अवधि',
 'सीमांत बोर्डर के रंग को हाइलाइट करें',
 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।',
 'भराई के रंग को हाइलाइट करें',
 'हाइलाइट किया गया भराई का रंग और पारदर्शिता।',
 'एपीआई विचरक',
 'इस समय जिसे प्राप्त किया गया हो उसकी विभिन्न विधियों मेथड में विचरण करें',
 'निजी गुणों को छिपाएं',
 'विधि',
 'गुणधर्म',
 'मान',
 'आईपाइथन कन्सोल',
 'इस समय चुने गए एक्सेसेबेल से काम लेने के लिए अंतर्क्रियात्मक कन्सोल',
 'घटना मानिटर',
 'घटनाओं को मानिटर करें ',
 'चुनाव को हटाएं ',
 'सभी',
 'चुने गए अनुप्रयोग',
 'चुने गए एक्सेसेबेल',
 'स्रोत',
 'घटना मानिटर',
 'चुने गए प्रकारों और स्रोतों से घटनाएं जैसेजैसे घटित होती हैं उन्हें दर्शाता है',
 'अंतिम प्रविष्ट घटना को हाइलाइट करो',
 'घटना 

In [58]:
len(en_data),len(hi_data)

(50000, 50000)

### Start and End Tags

In [59]:
## For decoder to understand when sentence starts and ends

hi_data = ['<START>' + hi + '<END>' for hi in hi_data]



In [60]:
hi_data

['<START>अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें<END>',
 '<START>एक्सेर्साइसर पहुंचनीयता अन्वेषक<END>',
 '<START>निचले पटल के लिए डिफोल्ट प्लगइन खाका<END>',
 '<START>ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका<END>',
 '<START>उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है<END>',
 '<START>अवधि को हाइलाइट रकें<END>',
 '<START>पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से की अवधि<END>',
 '<START>सीमांत बोर्डर के रंग को हाइलाइट करें<END>',
 '<START>हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।<END>',
 '<START>भराई के रंग को हाइलाइट करें<END>',
 '<START>हाइलाइट किया गया भराई का रंग और पारदर्शिता।<END>',
 '<START>एपीआई विचरक<END>',
 '<START>इस समय जिसे प्राप्त किया गया हो उसकी विभिन्न विधियों मेथड में विचरण करें<END>',
 '<START>निजी गुणों को छिपाएं<END>',
 '<START>विधि<END>',
 '<START>गुणधर्म<END>',
 '<START>मान<END>',
 '<START>आईपाइथन कन्सोल<END>',
 '<START>इस समय चुने गए एक्सेसेबेल से काम लेने के लिए अंतर्क्रियात्मक कन्सोल<END>',
 '<START>घटना मानिटर<END>',
 '<START>घटनाओं

## Tokenization

In [61]:
#Tokenize the texts and convert to sequences
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', lower=False)
en_tokenizer.fit_on_texts(en_data)
en_sequences = en_tokenizer.texts_to_sequences(en_data)

hi_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', lower=False)
hi_tokenizer.fit_on_texts(hi_data)
hi_sequences = hi_tokenizer.texts_to_sequences(hi_data)

english_vocab_size = len(en_tokenizer.word_index) + 1
hindi_vocab_size = len(hi_tokenizer.word_index) + 1
print("English Vocab Size: ", english_vocab_size)
print("Hindi Vocab Size: ", hindi_vocab_size)

English Vocab Size:  3503
Hindi Vocab Size:  5959


In [64]:
len(en_sequences),len(hi_sequences)

(50000, 50000)

In [70]:
en_sequences[:10]

[[1468, 83, 180, 24, 588, 1991],
 [952, 588, 1396],
 [2, 162, 20, 166, 9, 2, 498, 430],
 [2, 162, 20, 166, 9, 2, 321, 430],
 [6, 106, 4, 122, 107, 57, 705, 96, 162],
 [175, 289],
 [2, 289, 4, 2, 175, 335, 205, 1397, 129, 1720],
 [175, 470, 86],
 [2, 86, 18, 555, 4, 2, 175, 470],
 [175, 310, 86]]

In [71]:
hi_sequences[:10]

[[859, 327, 5, 689, 2738, 6, 2739, 194],
 [1125, 689, 1629],
 [4236, 789, 4, 10, 992, 134, 1455],
 [1126, 789, 4, 10, 992, 134, 1455],
 [1127, 1845, 11, 99, 1846, 992, 97, 13, 127, 33, 75, 7],
 [1503, 5, 626, 2740],
 [2741, 2742, 648, 5, 2743, 173, 626, 1128, 11, 444],
 [1630, 2744, 4, 192, 5, 626, 8],
 [537, 256, 288, 1631, 6, 192, 20, 2745],
 [1632, 4, 192, 5, 626, 8]]

## Encoder Inputs

In [62]:
#Prepare encoder data
encoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(en_sequences, maxlen=maxlen, padding='post')

In [66]:
encoder_inputs[:10]

array([[1468,   83,  180,   24,  588, 1991,    0,    0,    0,    0],
       [ 952,  588, 1396,    0,    0,    0,    0,    0,    0,    0],
       [   2,  162,   20,  166,    9,    2,  498,  430,    0,    0],
       [   2,  162,   20,  166,    9,    2,  321,  430,    0,    0],
       [   6,  106,    4,  122,  107,   57,  705,   96,  162,    0],
       [ 175,  289,    0,    0,    0,    0,    0,    0,    0,    0],
       [   2,  289,    4,    2,  175,  335,  205, 1397,  129, 1720],
       [ 175,  470,   86,    0,    0,    0,    0,    0,    0,    0],
       [   2,   86,   18,  555,    4,    2,  175,  470,    0,    0],
       [ 175,  310,   86,    0,    0,    0,    0,    0,    0,    0]])

## Decoder Input and Outputs

In [77]:
#Prepare decoder data
decoder_inputs = []
decoder_outputs = []
count = 0
for hi in hi_sequences:
    while count < 20:
        print("Original Input:",hi)
        decoder_inputs.append(hi[:-1])
        print("Decoder Input:",hi[:-1])
        decoder_outputs.append(hi[1:])
        print("Decoder Output:",hi[1:])
        count+=1
        print('\n')
    else:
        decoder_inputs.append(hi[:-1])
        decoder_outputs.append(hi[1:])
        continue
decoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_inputs, maxlen=maxlen, padding='post')
decoder_outputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_outputs, maxlen=maxlen, padding='post')

Original Input: [859, 327, 5, 689, 2738, 6, 2739, 194]
Decoder Input: [859, 327, 5, 689, 2738, 6, 2739]
Decoder Output: [327, 5, 689, 2738, 6, 2739, 194]


Original Input: [859, 327, 5, 689, 2738, 6, 2739, 194]
Decoder Input: [859, 327, 5, 689, 2738, 6, 2739]
Decoder Output: [327, 5, 689, 2738, 6, 2739, 194]


Original Input: [859, 327, 5, 689, 2738, 6, 2739, 194]
Decoder Input: [859, 327, 5, 689, 2738, 6, 2739]
Decoder Output: [327, 5, 689, 2738, 6, 2739, 194]


Original Input: [859, 327, 5, 689, 2738, 6, 2739, 194]
Decoder Input: [859, 327, 5, 689, 2738, 6, 2739]
Decoder Output: [327, 5, 689, 2738, 6, 2739, 194]


Original Input: [859, 327, 5, 689, 2738, 6, 2739, 194]
Decoder Input: [859, 327, 5, 689, 2738, 6, 2739]
Decoder Output: [327, 5, 689, 2738, 6, 2739, 194]


Original Input: [859, 327, 5, 689, 2738, 6, 2739, 194]
Decoder Input: [859, 327, 5, 689, 2738, 6, 2739]
Decoder Output: [327, 5, 689, 2738, 6, 2739, 194]


Original Input: [859, 327, 5, 689, 2738, 6, 2739, 194]
Decoder I

In [78]:
len(encoder_inputs),len(decoder_inputs),len(decoder_outputs)

(50000, 50020, 50020)

In [79]:
# Training and Testing split
# 95%, 5%
split = int(0.95 * total_sentences)

X_train = [encoder_inputs[:split], decoder_inputs[:split]]
y_train = decoder_outputs[:split]

# Test data to evaluate our NMT model using BLEU score
X_test = en_data[:split]
y_test = hi_data[:split]

print(X_train[0].shape, X_train[1].shape, y_train.shape)

(23750, 10) (23750, 10) (23750, 10)


In [84]:
#Define LSTM model
d_model = 256

#Encoder
inputs = tf.keras.layers.Input(shape=(None,))
x = tf.keras.layers.Embedding(english_vocab_size, d_model, mask_zero=True)(inputs)
_,state_h,state_c = tf.keras.layers.LSTM(d_model,activation='relu',return_state=True)(x)

#Decoder
targets = tf.keras.layers.Input(shape=(None,))
embedding_layer = tf.keras.layers.Embedding(hindi_vocab_size, d_model, mask_zero=True)
x = embedding_layer(targets)
decoder_lstm = tf.keras.layers.LSTM(d_model,activation='relu',return_sequences=True, return_state=True)
x,_,_ = decoder_lstm(x, initial_state=[state_h, state_c])
dense1 = tf.keras.layers.Dense(hindi_vocab_size, activation='softmax')
x = dense1(x)

model = tf.keras.models.Model(inputs=[inputs, targets],outputs=x)
model.summary()

loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])

NotImplementedError: Cannot convert a symbolic Tensor (lstm_2/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported